<a href="https://colab.research.google.com/github/AmbiTyga/Bio-VI-BERT/blob/main/Multi-label-Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title EarlyStopping
class EarlyStopping(object):
    def __init__(self, mode='min', min_delta=0, patience=10, percentage=False):
        self.mode = mode
        self.min_delta = min_delta
        self.patience = patience
        self.best = None
        self.num_bad_epochs = 0
        self.is_better = None
        self._init_is_better(mode, min_delta, percentage)

        if patience == 0:
            self.is_better = lambda a, b: True
            self.step = lambda a: False

    def step(self, metrics):
        if self.best is None:
            self.best = metrics
            return False

        if torch.isnan(metrics):
            return True

        if self.is_better(metrics, self.best):
            self.num_bad_epochs = 0
            self.best = metrics
        else:
            self.num_bad_epochs += 1

        if self.num_bad_epochs >= self.patience:
            return True

        return False

    def _init_is_better(self, mode, min_delta, percentage):
        if mode not in {'min', 'max'}:
            raise ValueError('mode ' + mode + ' is unknown!')
        if not percentage:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - min_delta
            if mode == 'max':
                self.is_better = lambda a, best: a > best + min_delta
        else:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - (
                            best * min_delta / 100)
            if mode == 'max':
                self.is_better = lambda a, best: a > best + (
                            best * min_delta / 100)

In [ ]:
!git clone https://github.com/rwightman/pytorch-image-models.git
%cd pytorch-image-models
!pip install -r requirements.txt -q
!python setup.py install -q
%cd ..

import os
os.kill(os.getpid(), 9)
# Restart Runtime

In [2]:
!wget https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Dataset.7z
!7z x /content/Dataset.7z

--2021-03-10 15:01:01--  https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Dataset.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17231696 (16M) [application/octet-stream]
Saving to: ‘Dataset.7z’

Dataset.7z          100%[===================>]  16.43M  77.1MB/s    in 0.2s    

2021-03-10 15:01:02 (77.1 MB/s) - ‘Dataset.7z’ saved [17231696/17231696]


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 17231696 bytes (17 MiB)

Extracting archive: /content/Dataset.7z
--
Path = /content/Dat

In [3]:
import timm
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader, sampler
from sklearn.preprocessing import LabelEncoder
import os
from sklearn.model_selection import train_test_split
from glob import glob
from PIL import Image
from torchvision import transforms

## Cleaning and getting a final data

In [4]:
## Getting images(file path) from the directories 

imgs = []
for path, subdirs, files in os.walk('./Dataset'):
    for name in files:
        imgs.append(os.path.join(path, name))
imgs = [x for x in imgs if '.csv' not in x]

In [5]:
data = pd.read_csv("/content/Dataset/all_meta_data.csv")
data.head()

,phylum,class,genus,species,form,sample,image_name,image_url,img_path
0,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_worm4_HB.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_worm4_HB.jpg
1,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_egg_HBa.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_egg_HBa.jpg
2,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_egg_wtmt.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_egg_wtmt.jpg
3,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_SC_egg.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_SC_egg.jpg
4,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_egg_UVa.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_egg_UVa.jpg


In [6]:
import urllib.request as req

def download(url,file_name):
  imgurl =url
  req.urlretrieve(imgurl, file_name)

In [7]:
# Dropping unnecessary datapoints
def check_file_download(x):
  if x[1] not in imgs:
    download(*x)


data['image_path_url'] = data[['image_url','img_path']].values.tolist()
data['image_path_url'] = data['image_path_url'].apply(check_file_download)

In [8]:
data.to_csv('/content/Parasitesv1.csv',index = False)

In [9]:
data = [x for _,x in data.groupby('species') if len(x)>15]

data = pd.concat(data,ignore_index=True)

In [10]:
le = LabelEncoder()
data['species_label'] = le.fit_transform(data['species'])

In [11]:
train, val = train_test_split(
    data, test_size=0.25, random_state=2021,stratify = data['species'])

In [12]:
train.to_csv('/content/train.csv',index = False)

val.to_csv('/content/val.csv',index = False)

# Dataset Function

In [13]:
class SpeciesDataset(Dataset):
  def __init__(self,csv_file,transform):
    super().__init__()
    csv = pd.read_csv(csv_file)[['species','img_path']]
    labels = csv['species'].values

    self.images = csv['img_path'].values
    self.transform = transform

    self.LE = LabelEncoder()
    self.labels = self.LE.fit_transform(labels)    

  def __len__(self):
    # return size of dataset
    return len(self.images)

  def __getitem__(self, index):
    img = Image.open(self.images[index])
    img = self.transform(img)

    label = self.labels[index]

    return img, torch.LongTensor([label])

In [14]:

train_transformer = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        # transforms.
        # transforms.ColorJitter(hue=.05, saturation=.05),
        transforms.RandomRotation(90),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

val_transformer = transforms.Compose([
        transforms.Resize((224,224)),
#         transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [15]:
val = pd.read_csv('/content/val.csv')
class_counts = val['species_label'].value_counts().values
weights = torch.tensor([1/class_counts[label] for label in val['species_label'].values])

In [34]:
9*16

144

In [16]:
train = pd.read_csv('/content/train.csv')
class_counts = train['species_label'].value_counts().values
weights = torch.tensor([1/class_counts[label] for label in train['species_label'].values])

train_dataset = SpeciesDataset('/content/train.csv',transform=train_transformer)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=48,
                                           sampler=torch.utils.data.WeightedRandomSampler(weights,num_samples=432))

val = pd.read_csv('/content/val.csv')
class_counts = val['species_label'].value_counts().values
weights = torch.tensor([1/class_counts[label] for label in val['species_label'].values])

val_dataset = SpeciesDataset('/content/val.csv',transform=val_transformer)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=17,
                                         sampler=torch.utils.data.WeightedRandomSampler(weights,num_samples=144))

# VIT Base Imagenet 21k

In [17]:
class ViT(nn.Module):
  def __init__(self,num_classes_classifier=16):
    super().__init__()
    self.img_transformer = timm.models.vision_transformer.vit_base_patch16_224_in21k(pretrained=True,num_classes = 512)
    self.drop = nn.Dropout(0.4)
    self.classifier = nn.Linear(512,num_classes_classifier)

  def forward(self,img):
    repr = self.img_transformer(img)
    repr = self.drop(repr)
    output = self.classifier(repr)

    return output  

In [18]:
model = ViT()

Removing representation layer for fine-tuning.
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_patch16_224_in21k-e5005f0a.pth" to /root/.cache/torch/hub/checkpoints/jx_vit_base_patch16_224_in21k-e5005f0a.pth


In [ ]:
model.load_state_dict(torch.load('vit-base.bin'))

<All keys matched successfully>

In [19]:
device = torch.device('cuda')
optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr = 1e-4)
loss_fn = nn.CrossEntropyLoss().to(device)
es = EarlyStopping(patience = 10)
es.best = np.inf

model.cuda()

train_losses = []
train_accs = []
val_losses = []
val_accs = []
for epoch in range(100):
  train_epoch_loss = 0
  train_epoch_acc = 0
  model.train()
  steps = 0
  for idx, batch in enumerate(train_loader):
    image, target = batch
    image = image.to(device)
    target = target.flatten().to(device)

    optim.zero_grad()

    prediction = model(image)
    # print(prediction,target)
    loss = loss_fn(prediction, target)

    num_corrects = (prediction.argmax(dim=1).view(target.size()).data == target.data).sum()
    acc = 100.0 * (num_corrects/target.size(0))
    loss.backward()
    optim.step()
    steps += 1
    
    # if steps % 6 == 0:
    #     print (f'\t Train - Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')
    
    train_epoch_loss += loss.item()
    train_epoch_acc += acc.item()

  val_epoch_loss = 0
  val_epoch_acc = 0
  model.eval()
  val_steps = 0
  with torch.no_grad():
    for idx, batch in enumerate(val_loader):
      image, target = batch
      image = image.to(device)
      target = target.flatten().to(device)

      prediction = model(image)
      loss = loss_fn(prediction, target)
      
      num_corrects = (prediction.argmax(dim=1).view(target.size()).data == target.data).sum()
      acc = 100.0 * (num_corrects/target.size(0))

      val_epoch_loss += loss.item()
      val_epoch_acc += acc.item()
      val_steps +=1

  
  val_epoch_loss /=val_steps
  val_epoch_acc /=val_steps

  # val_losses.append(val_epoch_loss)
  # val_accs.append(val_epoch_acc)
  
  print(f"Epoch: {epoch+1:02}, Train Loss: {train_epoch_loss/steps:.3f}, Train Acc: {train_epoch_acc/steps:.2f}%, Val. Loss: {val_epoch_loss:.3f}, Val. Acc: {val_epoch_acc:.2f}% \n{90*'='}")
  
  if val_epoch_loss < es.best:
    best_loss = val_epoch_loss
    print(f'\r\t{es.best:.3f} --> {best_loss:.3f}  Updating')
    torch.save(model.img_transformer.state_dict(),'vit-base.bin')

    train_losses.append(train_epoch_loss/steps)
    train_accs.append(train_epoch_acc/steps)

    val_losses.append(val_epoch_loss)
    val_accs.append(val_epoch_acc)
    
  if es.step(torch.tensor(val_epoch_loss)):
    print(f'\r\tPatience complete! Best Loss is {es.best:.3f}')
    break
  if val_epoch_loss > best_loss:
    print(f'\r\t Patience -> {es.patience - es.num_bad_epochs}')

Epoch: 01, Train Loss: 2.644, Train Acc: 24.07%, Val. Loss: 2.600, Val. Acc: 26.96% 
	inf --> 2.600  Updating
Epoch: 02, Train Loss: 2.362, Train Acc: 40.97%, Val. Loss: 2.407, Val. Acc: 33.58% 
	2.600 --> 2.407  Updating
Epoch: 03, Train Loss: 2.083, Train Acc: 57.18%, Val. Loss: 2.187, Val. Acc: 39.46% 
	2.407 --> 2.187  Updating
Epoch: 04, Train Loss: 1.810, Train Acc: 66.44%, Val. Loss: 1.987, Val. Acc: 49.26% 
	2.187 --> 1.987  Updating
Epoch: 05, Train Loss: 1.566, Train Acc: 69.68%, Val. Loss: 1.726, Val. Acc: 63.89% 
	1.987 --> 1.726  Updating
Epoch: 06, Train Loss: 1.272, Train Acc: 80.79%, Val. Loss: 1.595, Val. Acc: 57.03% 
	1.726 --> 1.595  Updating
Epoch: 07, Train Loss: 1.061, Train Acc: 84.95%, Val. Loss: 1.397, Val. Acc: 65.85% 
	1.595 --> 1.397  Updating
Epoch: 08, Train Loss: 0.826, Train Acc: 90.74%, Val. Loss: 1.403, Val. Acc: 65.60% 
	 Patience -> 9
Epoch: 09, Train Loss: 0.645, Train Acc: 95.83%, Val. Loss: 1.183, Val. Acc: 68.38% 
	1.397 --> 1.183  Updating
Epoch

# Multi-label Classification

In [83]:
class MultiDataset(Dataset):
  def __init__(self,dataFrame,transform=None):
    self.data = dataFrame
    self.transform = transform
    self.labels = ["phylum","class","species","form","sample"]
    self.label_encoder = self.get_label_encode()
    self.nclasses = self.get_nclasses()
    total = fsum(self.nclasses.values())

    self.weights = {k:v/total for k,v in self.nclasses.items()}

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    img_path = self.data.loc[idx,"img_path"]
    image = Image.open(img_path)
    image = self.transform(image)
    labels = self.data.loc[idx,self.labels].to_dict()
    for label in labels:
      labels[label] = torch.tensor(labels[label],dtype = torch.long)
            
    dict_data = {
    'image': image,
    'labels': labels
    }
    return dict_data
  
  def get_label_encode(self):
    d=dict()
    for label in self.labels:            
        le = LabelEncoder()
        self.data[label]=le.fit_transform(self.data[label])
        d[label]=le
    return d
    
  def get_nclasses(self):
    return self.data[self.labels].nunique().to_dict()

In [33]:
class MultiViT(nn.Module):
  def __init__(self,vit_path,nclasses,labels):
    super(MultiViT,self).__init__()


    self.img_transformer = timm.models.vision_transformer.vit_base_patch16_224_in21k(pretrained=False,num_classes = 512)
    self.img_transformer.load_state_dict(torch.load(vit_path))

    for param in self.img_transformer.parameters():
      param.requires_grad = False
    self.drop = nn.Dropout(0.38)
    self.classifiers = nn.ModuleDict()
    for label in labels:
      self.classifiers[label]=nn.Linear(512,nclasses[label])

    
  def forward(self,x):
    x=self.img_transformer(x)
    x=self.drop(x)
    y=dict()
    for label in self.classifiers.keys():
        y[label]=self.classifiers[label](x)
    # print(y.keys())
    return y


In [47]:
train = pd.read_csv("/content/train.csv")
val = pd.read_csv("/content/val.csv")

train_dataset = MultiDataset(train,train_transformer)
val_dataset = MultiDataset(val,val_transformer)

train_loader = DataLoader(train_dataset,batch_size=32)
val_loader = DataLoader(val_dataset,batch_size=32)

In [37]:
multimodel = MultiViT('/content/vit-base.bin',train_dataset.nclasses,train_dataset.labels)
multimodel.to(device)
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(filter(lambda p: p.requires_grad, multimodel.parameters()),lr = 1e-4)

Removing representation layer for fine-tuning.


In [75]:
from math import fsum

In [76]:
fsum([2,5])

7.0

In [77]:
fsum(train_dataset.nclasses.values())

50.0

In [65]:
7/50, 12/50, 5/50,10/50,16/50

1.0

In [81]:
X.items()

dict_items([('phylum', 5), ('class', 7), ('species', 16), ('form', 12), ('sample', 10)])

In [82]:
X = data[['phylum','class','species','form','sample']].nunique().to_dict()
{k:v/50 for k,v in X.items()}

{'class': 0.14, 'form': 0.24, 'phylum': 0.1, 'sample': 0.2, 'species': 0.32}

In [48]:
#@title Functions for training
def get_loss(out,labels,loss_fn):
    loss=0
    for label in labels:
        loss+=loss_fn(out[label],labels[label])
    return loss/len(labels)


def cal_accuracy(out,labels,batch_size,epoch_acc):
    for key in out:
        q=(out[key].detach().argmax(axis=1)==labels[key]).sum().item()/batch_size
        if not key in epoch_acc.keys():
            epoch_acc[key]=q
        else:
            epoch_acc[key]+=q
    return epoch_acc


def get_avg_acc(epoch_acc,loader):
    acc=0
    for key in epoch_acc:
        epoch_acc[key]/=len(loader)
        acc+=epoch_acc[key]
        
    return acc/len(epoch_acc),epoch_acc

def evaluate(model,loss_fn,loader):
    model.eval()
    epoch_acc={}
    epoch_loss=0
    for batch in loader:
        img=batch["image"].to(device)
        labels=batch["labels"]
        for key in labels:
            labels[key]=labels[key].to(device)
        
        out=model(img)
        loss=get_loss(out,labels,loss_fn)
        epoch_loss+=loss.detach().item()
        epoch_acc=cal_accuracy(out,labels,img.shape[0],epoch_acc)
        
    avg_acc,epoch_acc=get_avg_acc(epoch_acc,loader)
    return (epoch_loss/len(loader),avg_acc,epoch_acc)

def train_model(model, loss_fn, opt,trainloader,valloader, num_epochs=1):
    since = time.time()
    best_model=""
    max_acc=-999
    train={"loss":[],"avg":[],"individual":[]}
    val={"loss":[],"avg":[],"individual":[]}
    for epoch in range(num_epochs):
        print('Epoch {} of {}'.format(epoch+1, num_epochs))
        epoch_loss=0
        epoch_acc={}
        model.train()
        # Iterate over data.
        for i,data in enumerate(trainloader):
            opt.zero_grad()
            img=data["image"].to(device)
            labels=data["labels"]
            for key in labels:
                labels[key]=labels[key].to(device)
            

            out=model(img)
            # print(out,labels,sep = '\n\n')
            loss=get_loss(out,labels,loss_fn)
            epoch_loss+=loss.detach().item()
            loss.backward()
            opt.step()
            epoch_acc=cal_accuracy(out,labels,img.shape[0],epoch_acc)
            
        val_result=evaluate(model,loss_fn,valloader)
        avg_acc,epoch_acc=get_avg_acc(epoch_acc,trainloader)
        avg_loss=epoch_loss/len(trainloader)
        print('Train')
        print(f"Loss: {avg_loss}\nAccuracy: Each_label_acc->{epoch_acc}\n\tAvg acc->{avg_acc}\n")
        print("Validation")
        print(f"Loss: {val_result[0]}\nAccuracy: Each_label_acc->{val_result[2]}\n\tAvg acc->{val_result[1]}")
        print("-"*50)
        
        train["avg"].append(avg_acc)
        train["individual"].append(epoch_acc)
        train["loss"].append(avg_loss)
        
        val["avg"].append(val_result[1])
        val["individual"].append(val_result[2])
        val["loss"].append(val_result[0])
        if val_result[1]>max_acc:
            max_acc=val_result[1]
            best_model=copy.deepcopy(model)

        
       

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    return train,val,best_model,max_acc

In [50]:
import time,copy

In [55]:
for label in ['phylum','class','species','form','sample']:
  print(data[label].nunique())

5
7
16
12
10


In [51]:
train,val,best_model,best_acc=train_model(multimodel, loss_fn, optim,train_loader,val_loader,num_epochs=100)

Epoch 1 of 100
Train
Loss: 2.1561455965042113
Accuracy: Each_label_acc->{'phylum': 0.240625, 'class': 0.28125, 'species': 0.16875, 'form': 0.096875, 'sample': 0.178125}
	Avg acc->0.19312500000000002

Validation
Loss: 2.1023948192596436
Accuracy: Each_label_acc->{'phylum': 0.3828125, 'class': 0.3359375, 'species': 0.3984375, 'form': 0.0859375, 'sample': 0.2421875}
	Avg acc->0.2890625
--------------------------------------------------
Epoch 2 of 100
Train
Loss: 2.0733668327331545
Accuracy: Each_label_acc->{'phylum': 0.32083333333333336, 'class': 0.359375, 'species': 0.253125, 'form': 0.140625, 'sample': 0.28125}
	Avg acc->0.2710416666666667

Validation
Loss: 2.0434752702713013
Accuracy: Each_label_acc->{'phylum': 0.53125, 'class': 0.3671875, 'species': 0.515625, 'form': 0.109375, 'sample': 0.328125}
	Avg acc->0.3703125
--------------------------------------------------
Epoch 3 of 100
Train
Loss: 1.9898480296134948
Accuracy: Each_label_acc->{'phylum': 0.475, 'class': 0.5197916666666667, '